In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def get_metrics(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred)**(1/2)
    r2training = model.score(x_train, y_train)
    r2test = model.score(x_test, y_test)
    r2 = r2_score(y_test, y_pred)
    print(model)
    print('R2 of Training:', r2training)
    print('R2 of Test:', r2test)
    print('R2 Score:', r2)
    print('MSE:', mse)
    print('RMSE:', rmse)
    
linear = LinearRegression()
ridge = Ridge()
lasso = Lasso()
gbr = GradientBoostingRegressor()
rfr = RandomForestRegressor()
knn = KNeighborsRegressor()

In [2]:
df = pd.read_csv('/Users/joshuarkarnad/Desktop/Thinkful/Data/Important Data/WeaponDataV3.csv')

In [3]:
oss_cols = [c for c in df.columns if 'OSS' in c]
df['Total OSS'] = df[oss_cols].sum(axis=1)
print(df.shape)
df['Total OSS'].describe()

(24979, 138)


count     24979.000000
mean         83.966011
std         895.605375
min           0.000000
25%           2.000000
50%          16.000000
75%          64.000000
max      122550.000000
Name: Total OSS, dtype: float64

In [4]:
exp_cols = [c for c in df.columns if 'Expulsion' in c]
df['Total Expulsion'] = df[exp_cols].sum(axis=1)
print(df.shape)
df['Total Expulsion'].describe()

(24979, 139)


count    24979.000000
mean         1.139197
std          6.215564
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        632.000000
Name: Total Expulsion, dtype: float64

In [5]:
df['Weapon OSS Percent'] = df['Weapon OSS'] / df['Total OSS']
df.loc[~np.isfinite(df['Weapon OSS Percent']), 'Weapon OSS Percent'] = 0
       
df['Weapon Expulsion Percent'] = df['Weapon Expulsion'] / df['Total Expulsion']
df.loc[~np.isfinite(df['Weapon OSS Percent']), 'Weapon OSS Percent'] = 0

df['Any Possession of Knife'] = ((df['Possession of a Knife'] > 1).astype(int))
df = df.drop('Possession of a Knife', axis = 1)
df['More than 10 Law Enforcement Incidents'] = ((df['No. of Incidents Involving Local Law Enforcement'] > 10).astype(int))
df = df.drop('No. of Incidents Involving Local Law Enforcement', axis = 1)

df['Enrollment More Than 1000'] = ((df['Enrollment'] > 1000).astype(int))

df['Arrests More Than 5'] = ((df['Enrollment'] > 5).astype(int))

df['Weapon OSS Percent over 10'] = ((df['Weapon OSS Percent'] > .1).astype(int))
df['Weapon EXP Percent over 30'] = ((df['Weapon Expulsion Percent'] > .3).astype(int))

df['Severe Weapon Punishments'] = df['Weapon Expulsion'] + df['Weapon OSS']
df['Total OSS Expulsions'] = df['Total OSS'] + df['Total Expulsion']
df['Severe Weapon Punishments Percent'] = ((df['Severe Weapon Punishments'] / 
                                            df['Total OSS Expulsions']))
df.loc[~np.isfinite(df['Severe Weapon Punishments Percent']),
       'Severe Weapon Punishments Percent'] = 0
df['Dangerous School'] = ((df['Weapon Incidents'] > 40).astype(int))

In [6]:
df['Weapon Incidents'].describe()

count    24979.000000
mean         2.009648
std          5.968807
min          0.000000
25%          0.000000
50%          0.000000
75%          2.000000
max        655.000000
Name: Weapon Incidents, dtype: float64

In [7]:
df =df[df['Year'] < 2014]
df = df.replace([np.inf, -np.inf], 0)
df = df.fillna(0)
df_train, df_test = train_test_split(df, test_size=0.2)
X_train = df_train.loc[:, ~(df_train.columns).isin(['FollowingYearWeapons'])]
Y_train = df_train['FollowingYearWeapons']
X_test = df_test.loc[:, ~(df_test.columns).isin(['FollowingYearWeapons'])]
Y_test = df_test['FollowingYearWeapons']

In [38]:
get_metrics(linear, X_train, Y_train, X_test, Y_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
R2 of Training: 0.513075578019
R2 of Test: 0.522684137247
MSE: 8.10194323615
RMSE: 2.84639126547


In [39]:
get_metrics(ridge, X_train, Y_train, X_test, Y_test)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
R2 of Training: 0.513069621171
R2 of Test: 0.522831172807
MSE: 8.09944745955
RMSE: 2.84595282103


In [40]:
get_metrics(lasso, X_train, Y_train, X_test, Y_test)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
R2 of Training: 0.48009598753
R2 of Test: 0.489766553532
MSE: 8.66068518366
RMSE: 2.94290420905


In [41]:
get_metrics(gbr, X_train, Y_train, X_test, Y_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)
R2 of Training: 0.62289521291
R2 of Test: 0.524294146443
MSE: 8.07461499476
RMSE: 2.84158670372


In [42]:
get_metrics(rfr, X_train, Y_train, X_test, Y_test)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
R2 of Training: 0.893821803734
R2 of Test: 0.48250960082
MSE: 8.78386445242
RMSE: 2.96375850103


In [43]:
get_metrics(knn, X_train, Y_train, X_test, Y_test)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')
R2 of Training: 0.553690897235
R2 of Test: 0.341909079358
MSE: 11.1704129264
RMSE: 3.34221676831


In [29]:
# df.to_csv('/Users/joshuarkarnad/Desktop/Thinkful/Data/Important Data/WeaponDataV4.csv')